In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Notebooks/Github/twitter-sentiment-analysis/

/content/drive/MyDrive/Notebooks/Github/twitter-sentiment-analysis


In [4]:
import numpy as np
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm
tqdm.pandas()


In [5]:
tweet_files_path = os.getcwd() + '/dataset/Tweets/'
# https://www.kaggle.com/datasets/sohelranaccselab/stock-market-tweets-data-sentiment-analysis
tweets_df = pd.read_csv(tweet_files_path + 'tweets_remaining_09042020_16072020.csv', delimiter=';')
tweets_df.columns = ['id', 'date', 'tweets']
tweets_df = tweets_df[['date', 'tweets']]

In [6]:
# Null and duplicate check and removal
print(tweets_df.shape)
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
# print(tweets_df.duplicated().sum())
print(len(tweets_df)-len(tweets_df.drop_duplicates()))
tweets_df = tweets_df.drop_duplicates()
print(tweets_df.isna().sum())
print(tweets_df.shape)
tweets_df.sample(20)

(923673, 2)
date      0
tweets    0
dtype: int64
2113
date      0
tweets    0
dtype: int64
(921560, 2)


,date,tweets
812811,2020-07-09 18:27:51+00:00,$AMZn added more short
917042,2020-07-16 14:17:45+00:00,Entry Signal Time: 16/7 10:15\nBATS:SPCE\nAsse...
605711,2020-06-22 10:16:30+00:00,sp500 fut ... https://t.co/twJUSGm1pI
357629,2020-05-30 08:27:36+00:00,*ZUCKERBERG SAYS TRUMP POSTS DIDN'T VIOLATE FA...
285916,2020-05-06 22:25:27+00:00,RT @DanZanger: $FB Had to cover the most recen...
711122,2020-06-30 10:55:23+00:00,RT @RodG97: Annual dividends you receive from ...
128556,2020-04-20 14:27:13+00:00,I hate banks.. but like making $ &amp; playing...
165582,2020-04-23 20:00:08+00:00,"low priced scan $INO, $MR, $HEAR, $INSG #stock..."
419864,2020-06-05 14:33:15+00:00,RT @LlcBillionaire: $HDii I have a few lol we ...
296910,2020-05-06 03:37:53+00:00,Sprouts Farmers Market $SFM Receives “Buy” Rat...


In [7]:
# Due to incorrect formatting of the source csv files, some strings are available in date column. 
# So we are forcing those values to be NaT
# Also dropping those rows
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
print(tweets_df.shape)
tweets_df = tweets_df.reset_index(drop = True)

date      0
tweets    0
dtype: int64
(921560, 2)


In [8]:
# Wrote the cleaned dataset to a new csv file
tweets_df.to_csv('./dataset/Tweets/tweets-900k.csv', index = False)

In [9]:
# !pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.7 MB/s 
     |████████████████████████████████| 115 kB 89.2 MB/s 
     |████████████████████████████████| 163 kB 91.5 MB/s 
     |████████████████████████████████| 95 kB 6.6 MB/s 
     |████████████████████████████████| 212 kB 90.1 MB/s 
     |████████████████████████████████| 127 kB 85.9 MB/s 
     |████████████████████████████████| 115 kB 81.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [14]:
# from datasets import load_dataset
# Dataset is similar to pandas dataframe from Huggingface project, but it supports batch processing
from datasets import Dataset

dataset = Dataset.from_pandas(tweets_df)

# dataset = load_dataset('csv', data_files= os.getcwd() + '/dataset/Tweets/tweets-900k.csv', split='train')
dataset[100]

{'date': Timestamp('2020-04-09 23:40:54+0000', tz='UTC'),
 'tweets': '$SPX is at 2800 50% FiB resistance level. It will either go above it or fall to 2600 next week.\n\nPlace your bets. https://t.co/loleRIbJpJ'}

In [15]:
# !pip install -q -U transformers
# !pip install -U torch torchvision torchaudio
# !pip install -U emoji

     |████████████████████████████████| 5.5 MB 4.6 MB/s 
     |████████████████████████████████| 7.6 MB 76.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████▉         | 635.2 MB 130.2 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████    | 777.1 MB 124.2 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



tcmalloc: large alloc 1147494400 bytes == 0x65fa4000 @  0x7f30d81b1615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 890.2 MB 6.1 kB/s 
     |████████████████████████████████| 24.3 MB 78.9 MB/s 
     |████████████████████████████████| 4.2 MB 79.7 MB/s 
     |████████████████████████████████| 849 kB 83.6 MB/s 
     |████████████████████▌           | 356.4 MB 107.0 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 557.1 MB 10 kB/s 
     |████████████████████████████████| 317.1 MB 38 kB/s 
     |████████████████████████████████| 21.0 MB 75.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.13.0 which is incompatible.
Looking i

In [16]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [17]:
# https://huggingface.co/zhayunduo/roberta-base-stocktwits-finetuned?text=I+like+you.+I+love+you
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import emoji
import numpy as np
from scipy.special import softmax
import re

MODEL = f"zhayunduo/roberta-base-stocktwits-finetuned"
# config = AutoConfig.from_pretrained(MODEL)
tokenizer = RobertaTokenizer.from_pretrained(MODEL)
model = RobertaForSequenceClassification.from_pretrained(MODEL)
model.to(device)
device

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/782 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

device(type='cuda')

In [18]:
def preprocess_dataset(row):
  texts = row['tweets']
  # remove URLs
  texts = re.sub(r'https?://\S+', "", texts)
  texts = re.sub(r'www.\S+', "", texts)
  # remove '
  texts = texts.replace('&#39;', "'")
  # remove symbol names
  texts = re.sub(r'(\#)(\S+)', r'hashtag_\2', texts)
  texts = re.sub(r'(\$)([A-Za-z]+)', r'cashtag_\2', texts)
  # remove usernames
  texts = re.sub(r'(\@)(\S+)', r'mention_\2', texts)
  # demojize
  texts = emoji.demojize(texts, delimiters=("", " "))

  row['tweets'] = texts.strip()
  return row

In [19]:
preprocessed_dataset =  dataset.map(preprocess_dataset)

  0%|          | 0/921560 [00:00<?, ?ex/s]

In [20]:
def tokenize_function(row):
  return tokenizer(row['tweets'], return_tensors='pt', truncation=True, padding = 'max_length', max_length = 512).to(device)

In [21]:
tokenized_datasets = preprocessed_dataset.map(tokenize_function, batched=True)

  0%|          | 0/922 [00:00<?, ?ba/s]

In [22]:
len(tokenized_datasets)

921560

In [23]:
# I am changing the type of tensor columns (input_ids, attention_mask) to torch type
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')


In [24]:
dataloader = torch.utils.data.DataLoader(tokenized_datasets, batch_size=32)

In [ ]:
scores_dict = {
    'bearish': [],
    'bullish': []
}

for batch in tqdm(dataloader):
  # with torch.no_grad():        
  outputs = model(**batch)
  logits = outputs.logits
  logits = logits.detach().cpu().numpy()
  for logit in logits:
    scores = softmax(logit)
    scores_dict['bearish'].append(scores[0])
    scores_dict['bullish'].append(scores[1])



  0%|          | 0/28799 [00:00<?, ?it/s]

In [29]:
scores_df = pd.DataFrame(scores_dict)
print(scores_df.shape)
display(scores_df.head())

(921560, 2)


,bearish,bullish
0,0.814516,0.185484
1,0.001386,0.998614
2,0.028479,0.971521
3,0.080560,0.919440
4,0.004977,0.995023


In [30]:
len(tweets_df)

921560

In [ ]:
final_df = pd.concat([tweets_df, scores_df], axis = 1)
final_df.to_csv('./dataset/Tweets/tweets-sentiment-900k.csv', index = False)